In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import lifelines
import pycox
import xgboost as xgb
from sklearn.model_selection import train_test_split

from xgbse import XGBSEKaplanNeighbors, XGBSEKaplanTree, XGBSEDebiasedBCE, XGBSEBootstrapEstimator, XGBSEStackedWeibull
from xgbse.converters import convert_data_to_xgb_format, convert_to_structured, convert_y
from xgbse.non_parametric import get_time_bins

from benchmark import BenchmarkLifelines, BenchmarkXGBoost, BenchmarkXGBSE, BenchmarkPysurvival
#from pysurvival.models.survival_forest import ConditionalSurvivalForestModel

# setting seed
np.random.seed(42)

In [3]:
def split_train_test_valid(dataf, test_size=.2, valid_size=.01, random_state=1):
    df_train, df_test = train_test_split(dataf, test_size=test_size, random_state=random_state)
    df_train, df_valid = train_test_split(df_train, test_size=valid_size, random_state=random_state)
    return df_train, df_valid,  df_test

## Data

In [4]:
from pycox.datasets import support

In [5]:
support_df = support.read_df()
df_train, df_valid, df_test = split_train_test_valid(support_df)

T_train = df_train.duration
E_train = df_train.event
TIME_BINS = time_bins = get_time_bins(T_train, E_train, size=30)

Dataset 'support' not locally available. Downloading...
Done


## Model

Let us fit a model and check performance.

In [6]:
results = pd.DataFrame()

## Lifelines

In [7]:

bench_lifelines = BenchmarkLifelines(lifelines.CoxPHFitter(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "Cox-PH"
)

In [8]:
bench_lifelines.train()

In [9]:
cox_lifelines_results = bench_lifelines.test()
cox_lifelines_results

{'model': 'Cox-PH',
 'c-index': 0.5780217892329218,
 'ibs': 0.2013043483549938,
 'dcal_pval': 0.0,
 'dcal_max_dev': 0.16042116729003866,
 'training_time': 0.46547985076904297,
 'inference_time': 0.00558018684387207}

In [10]:
results = results.append(cox_lifelines_results, ignore_index=True)

In [11]:
bench_lifelines = BenchmarkLifelines(lifelines.WeibullAFTFitter(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "Weibull AFT"
)

In [12]:
bench_lifelines.train()
weibull_aft_results = bench_lifelines.test()
weibull_aft_results

{'model': 'Weibull AFT',
 'c-index': 0.5764595917903831,
 'ibs': 0.201478808775029,
 'dcal_pval': 5.912690299669512e-252,
 'dcal_max_dev': 0.13831859544149952,
 'training_time': 0.4608640670776367,
 'inference_time': 0.006687164306640625}

In [13]:
results = results.append(weibull_aft_results, ignore_index=True)

## XGBoost

In [14]:
bench_xgboost = BenchmarkXGBoost(xgb,
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGB - AFT",
        "survival:aft",
)

In [15]:
bench_xgboost.train()
xgboost_aft_results = bench_xgboost.test()
xgboost_aft_results

{'model': 'XGB - AFT',
 'c-index': 0.608615671766739,
 'ibs': nan,
 'dcal_pval': nan,
 'dcal_max_dev': nan,
 'training_time': 0.1367959976196289,
 'inference_time': 0.0008289813995361328}

In [16]:
results = results.append(xgboost_aft_results, ignore_index=True)

In [17]:
bench_xgboost = BenchmarkXGBoost(xgb,
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGB - Cox",
        "survival:cox",
)
bench_xgboost.train()
xgboost_cox_results = bench_xgboost.test()
xgboost_cox_results

{'model': 'XGB - Cox',
 'c-index': 0.6103962565498003,
 'ibs': nan,
 'dcal_pval': nan,
 'dcal_max_dev': nan,
 'training_time': 0.06884503364562988,
 'inference_time': 0.0008327960968017578}

In [18]:
results = results.append(xgboost_cox_results, ignore_index=True)

## XGB SE

In [19]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEDebiasedBCE(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Debiased BCE",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_bce_results = bench_xgboost_embedding.test()
xgboost_bce_results

{'model': 'XGBSE - Debiased BCE',
 'c-index': 0.6170781812119915,
 'ibs': 0.18848515920234526,
 'dcal_pval': 1.2695847564787107e-261,
 'dcal_max_dev': 0.13931966387559408,
 'training_time': 3.851815938949585,
 'inference_time': 0.27188897132873535}

In [20]:
results = results.append(xgboost_bce_results, ignore_index=True)

In [21]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEKaplanNeighbors(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Kaplan Neighbors",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_kn_results = bench_xgboost_embedding.test()
xgboost_kn_results

{'model': 'XGBSE - Kaplan Neighbors',
 'c-index': 0.600916790055514,
 'ibs': 0.19707657293093728,
 'dcal_pval': 1.4246814978646604e-123,
 'dcal_max_dev': 0.09887323943661973,
 'training_time': 0.7519700527191162,
 'inference_time': 1.4877333641052246}

In [22]:
results = results.append(xgboost_kn_results, ignore_index=True)

In [23]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEKaplanTree(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Kaplan Tree",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_kt_results = bench_xgboost_embedding.test()
xgboost_kt_results

{'model': 'XGBSE - Kaplan Tree',
 'c-index': 0.5976060379731908,
 'ibs': 0.20288464464793987,
 'dcal_pval': 1.360307803727658e-142,
 'dcal_max_dev': 0.0965224071702945,
 'training_time': 0.1490190029144287,
 'inference_time': 0.003683805465698242}

In [24]:
results = results.append(xgboost_kt_results, ignore_index=True)

In [25]:
base_tree = XGBSEKaplanTree()

bench_xgboost_embedding = BenchmarkXGBSE(
    XGBSEBootstrapEstimator(base_tree, n_estimators=100),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Bootstrap Trees",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_bootstrap_results = bench_xgboost_embedding.test()
xgboost_bootstrap_results

{'model': 'XGBSE - Bootstrap Trees',
 'c-index': 0.6072864208207579,
 'ibs': 0.18781453318160898,
 'dcal_pval': 2.711414876542169e-197,
 'dcal_max_dev': 0.10253653541171698,
 'training_time': 18.202492237091064,
 'inference_time': 0.3706178665161133}

In [26]:
results = results.append(xgboost_bootstrap_results, ignore_index=True)

In [27]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEStackedWeibull(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Stacked Weibull",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_weibull_results = bench_xgboost_embedding.test()
xgboost_weibull_results

{'model': 'XGBSE - Stacked Weibull',
 'c-index': 0.6209880425651486,
 'ibs': 0.19789744262262765,
 'dcal_pval': 1.1365179764288005e-136,
 'dcal_max_dev': 0.0922466248054489,
 'training_time': 0.9544377326965332,
 'inference_time': 0.012730121612548828}

In [28]:
results = results.append(xgboost_weibull_results, ignore_index=True)

In [29]:
results.sort_values("c-index", ascending=False).round(3)

,c-index,dcal_max_dev,dcal_pval,ibs,inference_time,model,training_time
8,0.621,0.092,0.0,0.198,0.013,XGBSE - Stacked Weibull,0.954
4,0.617,0.139,0.0,0.188,0.272,XGBSE - Debiased BCE,3.852
3,0.610,NaN,NaN,NaN,0.001,XGB - Cox,0.069
2,0.609,NaN,NaN,NaN,0.001,XGB - AFT,0.137
7,0.607,0.103,0.0,0.188,0.371,XGBSE - Bootstrap Trees,18.202
5,0.601,0.099,0.0,0.197,1.488,XGBSE - Kaplan Neighbors,0.752
6,0.598,0.097,0.0,0.203,0.004,XGBSE - Kaplan Tree,0.149
0,0.578,0.160,0.0,0.201,0.006,Cox-PH,0.465
1,0.576,0.138,0.0,0.201,0.007,Weibull AFT,0.461


In [30]:
results.sort_values("c-index", ascending=False).round(3).set_index('model').to_markdown()

'| model                    |   c-index |   dcal_max_dev |   dcal_pval |     ibs |   inference_time |   training_time |\n|:-------------------------|----------:|---------------:|------------:|--------:|-----------------:|----------------:|\n| XGBSE - Stacked Weibull  |     0.621 |          0.092 |           0 |   0.198 |            0.013 |           0.954 |\n| XGBSE - Debiased BCE     |     0.617 |          0.139 |           0 |   0.188 |            0.272 |           3.852 |\n| XGB - Cox                |     0.61  |        nan     |         nan | nan     |            0.001 |           0.069 |\n| XGB - AFT                |     0.609 |        nan     |         nan | nan     |            0.001 |           0.137 |\n| XGBSE - Bootstrap Trees  |     0.607 |          0.103 |           0 |   0.188 |            0.371 |          18.202 |\n| XGBSE - Kaplan Neighbors |     0.601 |          0.099 |           0 |   0.197 |            1.488 |           0.752 |\n| XGBSE - Kaplan Tree      |     0.598 